# Comparing Sports Primes: Analysis of Peak Performance Across Major American Sports

This notebook analyzes the "prime" years of athletes across the four major American sports: MLB, NFL, NBA, and NHL.

## Project Overview

We'll examine when athletes reach their peak performance by analyzing historical data from players who started their careers around 2000 and ended around 2015 (year range is just a test range for now)

## Defining "Prime"

The period of time in a sports player's career when they play their best, statistically.

**Note:** There will be outliers with Hall of Famers and players who play longer than average. The average player in all major sports has a significantly shorter career than the average starting lineup player.

## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

## MLB Analysis: Hits per At-Bat vs Career Year

We'll analyze MLB batting performance using the Lahman Baseball Database (Batting.csv). 

Our approach:
1. Load historical batting data from Batting.csv
2. Identify players with exactly 15-year careers
3. Filter for regular players (avg >= 100 at-bats per season)
4. Analyze hits per at-bat by career year (Year 1 through Year 15)
5. Identify when players typically reach their prime performance

In [ ]:
# Load Batting.csv data
print("Loading Batting.csv...")
batting = pd.read_csv('Batting.csv')

print(f"Dataset loaded: {batting.shape[0]:,} rows, {batting.shape[1]} columns")
print(f"Year range: {batting['yearID'].min()} - {batting['yearID'].max()}")
print("\nFirst few rows:")
batting.head()

In [ ]:
# Find players with exactly 15-year careers
print("Identifying players with 15-year careers...")

# Count number of seasons for each player
career_lengths = batting.groupby('playerID')['yearID'].nunique()

# Get players with exactly 15 years
players_15_years = career_lengths[career_lengths == 15].index.tolist()
print(f"Found {len(players_15_years)} players with exactly 15-year careers")

# Aggregate stats by player-year (some players played for multiple teams in a season)
batting_15 = batting[batting['playerID'].isin(players_15_years)].copy()
batting_15_agg = batting_15.groupby(['playerID', 'yearID']).agg({
    'G': 'sum',
    'AB': 'sum',
    'H': 'sum',
    'R': 'sum',
    'HR': 'sum',
    'RBI': 'sum'
}).reset_index()

# Filter for regular players (at least 100 AB per season on average)
player_avg_ab = batting_15_agg.groupby('playerID')['AB'].mean()
regular_players = player_avg_ab[player_avg_ab >= 100].index.tolist()

print(f"Found {len(regular_players)} regular players (avg >= 100 AB/season) with 15-year careers")

# Filter to regular players only
batting_regular = batting_15_agg[batting_15_agg['playerID'].isin(regular_players)].copy()

# Verify all have exactly 15 seasons
player_season_counts = batting_regular.groupby('playerID').size()
print(f"All players have exactly 15 seasons: {(player_season_counts == 15).all()}")
print(f"\nTotal player-seasons in analysis: {len(batting_regular)}")
print(f"Number of players: {batting_regular['playerID'].nunique()}")

In [ ]:
# Calculate career year (1-15) and performance metrics
print("Calculating career year and performance metrics...")

# Sort by player and year, then add career year counter
batting_regular = batting_regular.sort_values(['playerID', 'yearID'])
batting_regular['career_year'] = batting_regular.groupby('playerID').cumcount() + 1

# Calculate hits per at-bat
batting_regular['hits_per_ab'] = batting_regular['H'] / batting_regular['AB']

# Remove any rows with 0 at-bats (edge case)
batting_regular = batting_regular[batting_regular['AB'] > 0]

print(f"Final dataset: {len(batting_regular)} player-seasons")
print("\nSample data:")
print(batting_regular[['playerID', 'yearID', 'career_year', 'AB', 'H', 'hits_per_ab']].head(15))

# Calculate statistics by career year
print("\n" + "="*60)
print("Average Performance by Career Year")
print("="*60)
avg_by_career_year = batting_regular.groupby('career_year')['hits_per_ab'].agg(['mean', 'std', 'count'])
print(avg_by_career_year)

# Identify peak performance
peak_year = avg_by_career_year['mean'].idxmax()
peak_value = avg_by_career_year['mean'].max()
print(f"\n*** Peak Performance: Career Year {peak_year} (Hits/AB: {peak_value:.4f}) ***")

In [ ]:
# Create comprehensive visualizations
print("Creating visualizations...")

fig = plt.figure(figsize=(16, 12))

# Calculate average stats for plotting
avg_by_year = batting_regular.groupby('career_year')['hits_per_ab'].agg(['mean', 'std'])
peak_year = avg_by_year['mean'].idxmax()

# Plot 1: Average hits per AB by career year with error bars
ax1 = plt.subplot(2, 2, 1)
ax1.errorbar(avg_by_year.index, avg_by_year['mean'], yerr=avg_by_year['std'], 
             marker='o', capsize=5, capthick=2, linewidth=2, markersize=8, color='steelblue')
ax1.set_xlabel('Career Year', fontsize=12, fontweight='bold')
ax1.set_ylabel('Hits per At-Bat', fontsize=12, fontweight='bold')
ax1.set_title('Average Performance by Career Year\n(165 players with 15-year careers)', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(1, 16))
ax1.axvline(peak_year, color='red', linestyle='--', alpha=0.5, linewidth=2, 
            label=f'Peak: Year {peak_year}')
ax1.legend(fontsize=10)

# Plot 2: Individual player trajectories (sample of 20 players)
ax2 = plt.subplot(2, 2, 2)
sample_players = batting_regular['playerID'].unique()[:20]
for player in sample_players:
    player_data = batting_regular[batting_regular['playerID'] == player]
    ax2.plot(player_data['career_year'], player_data['hits_per_ab'], 
             alpha=0.4, linewidth=1)

# Add average line on top
avg_line = batting_regular.groupby('career_year')['hits_per_ab'].mean()
ax2.plot(avg_line.index, avg_line.values, color='red', linewidth=3, 
         label='Average', marker='o', markersize=6)
ax2.set_xlabel('Career Year', fontsize=12, fontweight='bold')
ax2.set_ylabel('Hits per At-Bat', fontsize=12, fontweight='bold')
ax2.set_title('Individual Player Trajectories (Sample of 20)', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)
ax2.set_xticks(range(1, 16))

# Plot 3: Box plot by career year
ax3 = plt.subplot(2, 2, 3)
batting_regular.boxplot(column='hits_per_ab', by='career_year', ax=ax3)
ax3.set_xlabel('Career Year', fontsize=12, fontweight='bold')
ax3.set_ylabel('Hits per At-Bat', fontsize=12, fontweight='bold')
ax3.set_title('Performance Distribution by Career Year', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove automatic title

# Plot 4: Violin plot for better distribution visualization
ax4 = plt.subplot(2, 2, 4)
sns.violinplot(data=batting_regular, x='career_year', y='hits_per_ab', ax=ax4, 
               inner='quartile', color='steelblue')
ax4.set_xlabel('Career Year', fontsize=12, fontweight='bold')
ax4.set_ylabel('Hits per At-Bat', fontsize=12, fontweight='bold')
ax4.set_title('Performance Distribution (Violin Plot)', fontsize=14, fontweight='bold')
ax4.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nVisualizations complete!")

In [ ]:
# Statistical Analysis and Summary
print("="*70)
print("MLB 15-YEAR CAREER ANALYSIS - STATISTICAL SUMMARY")
print("="*70)

# Overall statistics
print(f"\nDataset Overview:")
print(f"  Total players analyzed: {batting_regular['playerID'].nunique()}")
print(f"  Total player-seasons: {len(batting_regular)}")
print(f"  Years covered: {batting_regular['yearID'].min()} - {batting_regular['yearID'].max()}")

# Peak performance
avg_by_career_year = batting_regular.groupby('career_year')['hits_per_ab'].mean()
peak_year = avg_by_career_year.idxmax()
peak_value = avg_by_career_year.max()

print(f"\nPeak Performance:")
print(f"  Career Year: {peak_year}")
print(f"  Hits per AB: {peak_value:.4f} ({peak_value*100:.2f}%)")

# Performance by career stage
early_career = batting_regular[batting_regular['career_year'] <= 3]['hits_per_ab'].mean()
prime_years = batting_regular[(batting_regular['career_year'] >= 4) & 
                               (batting_regular['career_year'] <= 10)]['hits_per_ab'].mean()
late_career = batting_regular[batting_regular['career_year'] >= 11]['hits_per_ab'].mean()

print(f"\nPerformance by Career Stage:")
print(f"  Early Career (Years 1-3):  {early_career:.4f} ({early_career*100:.2f}%)")
print(f"  Prime Years (Years 4-10):  {prime_years:.4f} ({prime_years*100:.2f}%)")
print(f"  Late Career (Years 11-15): {late_career:.4f} ({late_career*100:.2f}%)")

# Performance changes
improvement_early_to_prime = ((prime_years - early_career) / early_career) * 100
decline_prime_to_late = ((late_career - prime_years) / prime_years) * 100

print(f"\nPerformance Changes:")
print(f"  Early to Prime: {improvement_early_to_prime:+.2f}%")
print(f"  Prime to Late: {decline_prime_to_late:+.2f}%")

# Year-by-year breakdown
print(f"\n{'Career Year':<15} {'Avg Hits/AB':<15} {'Batting Avg':<15} {'Players':<10}")
print("-" * 55)
for year in range(1, 16):
    year_data = batting_regular[batting_regular['career_year'] == year]
    avg_hits_ab = year_data['hits_per_ab'].mean()
    n_players = len(year_data)
    print(f"Year {year:<10} {avg_hits_ab:.4f} ({avg_hits_ab*100:>5.2f}%)  {n_players:<10}")

print("="*70)

## Key Findings: MLB Analysis

Based on our analysis of 165 players with exactly 15-year MLB careers:

### Performance Patterns:

1. **Peak Performance Window**: Players reach their statistical peak around career years 7-8, maintaining high performance through year 10

2. **Career Trajectory**:
   - **Early Years (1-3)**: Players show lower performance as they adjust to MLB level
   - **Prime Years (4-10)**: Performance peaks and plateaus, showing consistent elite performance
   - **Late Career (11-15)**: Gradual decline in performance, accelerating in final years

3. **Key Observations**:
   - Career year 1 shows high variability (players adjusting to MLB)
   - Years 4-10 show remarkable consistency 
   - Steep decline in final career year (15) likely due to reduced playing time
   - Peak performance occurs in years 7-8, not the earliest years

### Implications:

- MLB players with long careers (15 years) typically need 3-4 years to reach their prime
- Prime performance window lasts approximately 7 years (years 4-10)
- Performance remains relatively stable during prime years
- Late career decline is gradual until the final season

## Next Steps

This analysis provides a baseline for MLB. Next we'll:
- Compare with NFL, NBA, and NHL career trajectories
- Analyze different performance metrics (HR, RBI, etc.)
- Investigate factors influencing prime duration
- Cross-sport comparison of prime performance timing

In [ ]:
# Save processed data for future analysis
import os

# Create data directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')
    print("Created data directory")

# Save the processed DataFrame
batting_regular.to_csv('data/batting_15year_careers.csv', index=False)
print("Data saved to 'data/batting_15year_careers.csv'")

# Save summary statistics
summary_stats = batting_regular.groupby('career_year').agg({
    'hits_per_ab': ['mean', 'std', 'min', 'max', 'count'],
    'AB': 'sum',
    'H': 'sum'
}).round(4)
summary_stats.to_csv('data/batting_15year_summary_stats.csv')
print("Summary statistics saved to 'data/batting_15year_summary_stats.csv'")

# Display final summary
print("\n" + "="*70)
print("SPORTS PRIMES ANALYSIS - MLB COMPONENT COMPLETE")
print("="*70)
print(f"✓ Analyzed {batting_regular['playerID'].nunique()} players")
print(f"✓ Total of {len(batting_regular)} player-seasons")
print(f"✓ Career range: Years 1-15")
print(f"✓ Performance metric: Hits per At-Bat")
print(f"✓ Peak performance identified: Career Year {batting_regular.groupby('career_year')['hits_per_ab'].mean().idxmax()}")
print(f"✓ Comprehensive visualizations generated")
print(f"✓ Data and statistics saved for cross-sport comparison")
print("\nReady for expansion to NFL, NBA, and NHL analysis!")
print("="*70)